In [10]:
# Load the libraries
library(knitr)
library(limma)
library(minfi)
library(IlluminaHumanMethylationEPICv2manifest)
library(IlluminaHumanMethylationEPICv2anno.20a1.hg38)
library(IlluminaHumanMethylationEPICanno.ilm10b4.hg19)
library(RColorBrewer)
library(missMethyl)
library(minfiData)
library(Gviz)
library(DMRcate)
library(stringr)
library(ChAMP)
library(org.Hs.eg.db)



Attaching package: 'limma'


The following object is masked from 'package:BiocGenerics':

    plotMA



Attaching package: 'IlluminaHumanMethylationEPICanno.ilm10b4.hg19'


The following objects are masked from 'package:IlluminaHumanMethylationEPICv2anno.20a1.hg38':

    Islands.UCSC, Locations, Manifest, Other, SNPs.141CommonSingle,
    SNPs.142CommonSingle, SNPs.144CommonSingle, SNPs.146CommonSingle,
    SNPs.147CommonSingle, SNPs.Illumina


Loading required package: IlluminaHumanMethylation450kanno.ilmn12.hg19


Attaching package: 'IlluminaHumanMethylation450kanno.ilmn12.hg19'


The following objects are masked from 'package:IlluminaHumanMethylationEPICanno.ilm10b4.hg19':

    Islands.UCSC, Locations, Manifest, Other, SNPs.132CommonSingle,
    SNPs.135CommonSingle, SNPs.137CommonSingle, SNPs.138CommonSingle,
    SNPs.141CommonSingle, SNPs.142CommonSingle, SNPs.144CommonSingle,
    SNPs.146CommonSingle, SNPs.147CommonSingle, SNPs.Illumina


The following objects are masked from 'pac

In [18]:
res_path <- "../Intermediate_results"
# set up a path to the data directory
dataDirectory <- '../Dataset/Izac_Methylomics/FIN13296/iDAT'

# read in the sample sheet for the experiment
targets <- read.metharray.sheet(dataDirectory, pattern="Sample_Sheet_Diagnosis.csv")

# read in the raw data from the IDAT files
rgSet <- read.metharray.exp(targets=targets)

# Load annotation
annEPICv2 <- getAnnotation(IlluminaHumanMethylationEPICv2anno.20a1.hg38)

# give the samples descriptive names
targets$ID <- targets$Sample_Name
sampleNames(rgSet) <- targets$ID



[read.metharray.sheet] Found the following CSV files:



 [1] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/Sample_Sheet_Diagnosis.csv"
 [2] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760037/Sample_Sheet_Diagnosis.csv"
 [3] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760097/Sample_Sheet_Diagnosis.csv"
 [4] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760106/Sample_Sheet_Diagnosis.csv"
 [5] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760108/Sample_Sheet_Diagnosis.csv"
 [6] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760117/Sample_Sheet_Diagnosis.csv"
 [7] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760119/Sample_Sheet_Diagnosis.csv"
 [8] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760120/Sample_Sheet_Diagnosis.csv"
 [9] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760121/Sample_Sheet_Diagnosis.csv"
[10] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760123/Sample_Sheet_Diagnosis.csv"
[11] "../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760128/Sample_Sheet_Diagnosis.csv"
[12] "../Dataset/Izac

# QC

In [19]:
# calculate the detection p-values
detP <- detectionP(rgSet)

# remove poor quality samples
keep <- colMeans(detP) <= 0.02
rgSet <- rgSet[, keep]

# remove poor quality samples from targets data
targets <- targets[keep, ]

# remove poor quality samples from detection p-value table
detP <- detP[, keep]

# normalize the data; this results in a GenomicRatioSet object
mSetSn <- preprocessFunnorm(rgSet) 


[preprocessFunnorm] Background and dye bias correction with noob

[preprocessFunnorm] Mapping to genome

[preprocessFunnorm] Quantile extraction

Warning message in .getSex(CN = CN, xIndex = xIndex, yIndex = yIndex, cutoff = cutoff):
"An inconsistency was encountered while determining sex. One possibility is that only one sex is present. We recommend further checks, for example with the plotSex function."
[preprocessFunnorm] Normalization



In [20]:
# ensure probes are in the same order in the mSetSq and detP objects
detP <- detP[match(featureNames(mSetSn),rownames(detP)),] 

# remove any probes that have failed in one or more samples
keep <- rowSums(detP < 0.01) == ncol(mSetSn) 
mSetSnFlt <- mSetSn[keep, ]

# if your data includes males and females, remove probes on the sex chromosomes
keep <- !(featureNames(mSetSnFlt) %in% annEPICv2$Name[annEPICv2$chr %in% 
                                                        c("chrX","chrY")])
mSetSnFlt <- mSetSnFlt[keep,]

# remove probes with SNPs at CpG site
mSetSnFlt <- dropLociWithSnps(mSetSnFlt)

bVals <- getBeta(mSetSnFlt)
head(bVals[, 1:5])


,I_D_102,I_D_103,I_D_104,I_D_105,I_D_106
cg00381604_BC11,0.02506614,0.02738411,0.0258803,0.02514083,0.02645971
cg21870274_BC21,0.02902466,0.29048458,0.5270121,0.13674814,0.52941273
cg00002271_BC21,0.56381436,0.13428840,0.3561911,0.52883263,0.47953919
cg00004581_TC21,0.38983669,0.42059204,0.3882163,0.25960807,0.33045484
cg09261435_BC21,0.18146340,0.13962648,0.1447528,0.13627775,0.13958023
cg00005473_TC21,0.92029837,0.92830399,0.9060129,0.92814239,0.88103126


In [23]:
mset <- preprocessRaw(rgSet)

In [25]:
intensity <-  minfi::getMeth(mset) + minfi::getUnmeth(mset)
head(intensity)

,I_D_102,I_D_103,I_D_104,I_D_105,I_D_106,I_D_107,I_D_108,I_D_109,I_D_050,I_D_051,...,I_D_039,I_D_040,I_D_067,I_D_069,I_D_070,I_D_071,I_D_074,I_D_076,I_D_078,I_D_079
cg25324105_BC11,24561,26240,26267,33085,31854,28603,31051,34291,21268,23085,...,27803,27854,27951,28057,29077,30023,29625,29884,31342,33948
cg25383568_TC11,5191,9549,8059,11405,9788,11925,9495,11689,6464,10828,...,8474,12576,9204,7307,14636,11050,11873,11421,11918,9366
cg25455143_BC11,21815,23271,22541,24476,25308,23492,25697,31720,17660,20603,...,23521,24151,21681,19859,24816,27159,25977,24128,21715,25727
cg25459778_BC11,16490,16261,14178,16875,16037,17960,17728,18464,12269,13496,...,16543,19159,15300,17194,17846,19573,20336,19898,21523,19098
cg25487775_BC11,18865,14293,16120,17720,15584,15135,21675,20728,15790,14890,...,18065,13103,16729,15401,17408,16373,14715,14698,11883,9900
cg25595446_BC11,10378,12976,10368,17422,16108,15921,15389,15562,9215,10031,...,11094,14691,12287,11456,14690,14743,16044,17486,14351,15697


In [27]:
head(targets)

,Sample_ID,Sample_Well,Sample_Plate,Sample_Name,Pool_ID,Sample_Group,I.D.Code,Sample,Sample.Type,Tumour.Normal,Collection,Institution,Array,Slide,Basename,ID
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,FIN13295A89,A12,207881760004_R01C01_FIN13295A89_A12,I_D_102,NA,Diffuse midline glioma,I-D-102,SU-DIPG-17,Cell-line,Tumour,Post-mortem,Stanford Univeristy,R01C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R01C01/207881760004_R01C01,I_D_102
2,FIN13295A90,B12,207881760004_R02C01_FIN13295A90_B12,I_D_103,NA,Diffuse midline glioma,I-D-103,RA055,Cell-line,Tumour,Progression,Zero Childhood Cancer,R02C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R02C01/207881760004_R02C01,I_D_103
3,FIN13295A91,C12,207881760004_R03C01_FIN13295A91_C12,I_D_104,NA,Diffuse midline glioma,I-D-104,PBT-28,Cell-line,Tumour,Diagnosis,Seattle Children's Hospital,R03C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R03C01/207881760004_R03C01,I_D_104
4,FIN13295A92,D12,207881760004_R04C01_FIN13295A92_D12,I_D_105,NA,Diffuse midline glioma,I-D-105,P002306,Cell-line,Tumour,Unknown,Zero Childhood Cancer,R04C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R04C01/207881760004_R04C01,I_D_105
5,FIN13295A93,E12,207881760004_R05C01_FIN13295A93_E12,I_D_106,NA,Diffuse midline glioma,I-D-106,P007601,Cell-line,Tumour,Unknown,Zero Childhood Cancer,R05C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R05C01/207881760004_R05C01,I_D_106
6,FIN13295A94,F12,207881760004_R06C01_FIN13295A94_F12,I_D_107,NA,Diffuse midline glioma,I-D-107,P005401 Progression,Cell-line,Tumour,Progression,Zero Childhood Cancer,R06C01,207881760004,../Dataset/Izac_Methylomics/FIN13296/iDAT/207881760004/207881760004_R06C01/207881760004_R06C01,I_D_107


In [31]:
myCNA <- champ.CNA(intensity=intensity, controlGroup= "Cell-line", pheno=targets$Sample.Type, arraytype="EPIC")


[===========================]

[<<<<< ChAMP.CNA START >>>>>]

-----------------------------

champ.CNA Results will be saved in ./CHAMP_CNA .


ChaMP.CNA does not provide batch Correct on intensity data now, but you can use champ.runCombat to correct slides batch yourself.

<< Create Control Data >>

<< Calculate mean value difference between each sample to mean control samples >>

